# Reproducing Adam CDA Study (2018)
## Looking at Channels

In [2]:
import numpy as np
import glob
import pickle
import pandas as pd
import copy
import scipy

import mne
from mne import Epochs, pick_channels, find_events
from mne.io import read_raw_eeglab

import os
import sys
#sys.path.append('/Users/nick/Documents/PhD/CLCT/reprod/studies/')
sys.path.append('D:\\PhD\\CDA-review\\studies\\')
from cda_utils import get_epochs, get_CDA, prep_report, fill_report, get_report, checksum, add_cda_report, get_CDA_perf_report
#sys.path.append('/Users/nick/Documents/PhD/CLCT/reprod/studies/Adam-2018/')
sys.path.append('D:\\PhD\\CDA-review\\studies\\Adam-2018')
from events import get_specific_events, convert_triggers
 
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Prep Data

In [3]:
#==================================================================
# General Variables
#==================================================================
study_id = 'Adam2018-Exp1'

# Organize Triggers
triggers = dict();

# --------------------------
# Fuck-up with events when loading in Python. I don't know why... NEED TO CONVERT.
triggers['1'] = [31, 41]
triggers['3'] = [33, 43]
triggers['6'] = [36, 46]

triggers['left'] = [31, 33, 36]
triggers['right'] = [41, 43, 46]

triggers['good'] = [81,82,83,84,85,86]
triggers['bad'] = [80]

triggers['acc_0'] = [80]
triggers['acc_1'] = [81]
triggers['acc_2'] = [82]
triggers['acc_3'] = [83]
triggers['acc_4'] = [84]
triggers['acc_5'] = [85]
triggers['acc_6'] = [86]
    
# Labels
conds = ['1', '3', '6']
sides = ['left', 'right']
perfs = ['good', 'bad'] # Here it is slightly different! (because of granular accuracies!)
accs =  ['acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_4', 'acc_5', 'acc_6']

# Internal Triggers to Epochs Properly
internal_triggers = dict()
internal_triggers_count = 0
for cond in conds:
    for side in sides:
        for perf in accs: #perfs:
            internal_triggers_count = internal_triggers_count + 1
            internal_triggers['{}-{}-{}'.format(cond,side,perf)] = internal_triggers_count
            
# CDA Channels
chan_right = ['O2', 'P4', 'PO4', 'T6'] # Not using: 'OR' because of cleaning... (no position set for OR)
chan_left =  ['O1', 'P3', 'PO3', 'T5'] # Not using: 'OL' because of cleaning... (no position set for OL)

# Channels
chanLabels = {'EEG 000':'PO3', 'EEG 001':'PO4', 'EEG 002':'F3', 'EEG 003':'F4', 'EEG 004':'C3', 'EEG 005':'C4', 'EEG 006':'P3', 'EEG 007':'P4', 'EEG 008':'O1', 'EEG 009':'O2', 'EEG 010':'OL', 'EEG 011':'OR', 'EEG 012':'T3', 'EEG 013':'T4', 'EEG 014':'T5', 'EEG 015':'T6', 'EEG 016':'POz', 'EEG 017':'Cz', 'EEG 018':'Fz', 'EEG 019':'Pz', 'EEG 020':'VEOG', 'EEG 021':'HEOG'}

# Epoc Information
epoch_length = 1.5  #1.65
epoch_tmin = -0.2
baseline_corr = (-0.2, 0)

# CDA info
cda_window = (0.4, 1.5)

# CDA main variable!
cda_dict_all = dict()
epochs_dict_all = dict()
perf_report_all = dict()
report = prep_report(conds, sides, accs) #perfs)

times = None


In [3]:
#==================================================================
# Main Loop - Loading All Files & Extract CDAs.
#==================================================================
#directory_path = '/Volumes/Nick - Data/CDA Datasets/Adam 2018/Experiment 1/Raw EEG Files (.rdf)/YR EEGLab Files/'
directory_path = 'G:\\CDA Datasets\Adam 2018\\Experiment 1\\Raw EEG Files (.rdf)\\YR EEGLab Files\\'

eegfiles = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(directory_path)
    for f in files if f.endswith('.set')]

sep = '\\'
#filepath = directory_path + '1_DWR_Bilateral.set'
for filepath in eegfiles:
    filename = filepath[filepath.rfind(sep)+1:]
    print('Processing: ' + filename + '...')

    # ------------------------
    # 1. -- Load File --
    # ------------------------
    raw = read_raw_eeglab(filepath, preload=True)

    # ------------------------
    # 2. -- Preprocessing --
    # ------------------------
    # Set Electrodes Locations - Used Brain Products (locations seem to in eeg file...)
    raw.rename_channels(chanLabels);
    raw.set_channel_types({'VEOG':'eog', 'HEOG':'eog', 'OR':'misc', 'OL':'misc'})
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, on_missing='warn')

    preproc = raw

    # Filter
    preproc = preproc.filter(.1, 20., fir_design='firwin', skip_by_annotation='edge')

    # Events
    events = mne.events_from_annotations(preproc)

    # Select Channels (to save time on cleaning)
    #preproc = preproc.pick_channels(chan_right + chan_left)

    # Plot (visual check)
    #preproc.plot(events=events, block=True, lowpass=40)

    # ------------------------
    # 3. -- Epochs --
    # ------------------------
    new_triggers = copy.deepcopy(triggers)
    new_triggers = convert_triggers(events, conds, sides, perfs, accs, new_triggers)

    specific_events = get_specific_events(events[0], conds, sides, perfs, accs, new_triggers, internal_triggers)

    epochs_dict = get_epochs(preproc, specific_events, epoch_length, epoch_tmin, baseline_corr, clean=False)

    epochs_dict_all[filename] = epochs_dict

    # times, used for CDA mean amp report.
    if times is None: times = epochs_dict[conds[0]][sides[0]][accs[0]].times 
        
    # ------------------------
    # 6. - Report & Checksum -
    # ------------------------
    report = fill_report(report, filename, specific_events)
    #report = add_cda_report(report, filename, conds, sides, accs, cda_dict_all[filename], times, cda_window=cda_window)
    report.at[filename, 'checksum'] = checksum(specific_events, epochs_dict)

    perf_report_all[filename] = get_CDA_perf_report(epochs_dict, conds, sides, accs, chan_right, chan_left, cda_window=cda_window)

# Print report
print('{} Subjects & {} Trials'.format(len(report), 2*np.sum([report[cond].sum() for cond in conds])))
report

Processing: 10_DWR_Bilateral.set...
Reading G:\CDA Datasets\Adam 2018\Experiment 1\Raw EEG Files (.rdf)\YR EEGLab Files\10_DWR_Bilateral.fdt
Reading 0 ... 1847167  =      0.000 ...  7388.668 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 8251 samples (33.004 sec)

Used Annotations descriptions: ['101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '1

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


0 bad epochs dropped
A total of 688 epochs were extracted and baseline corrected.
---------- 1-left-acc_0 -------------
---------- 1-left-acc_1 -------------
---------- 1-left-acc_2 -------------
---------- 1-left-acc_3 -------------
---------- 1-left-acc_4 -------------
---------- 1-left-acc_5 -------------
---------- 1-left-acc_6 -------------
---------- 1-right-acc_0 -------------
---------- 1-right-acc_1 -------------
---------- 1-right-acc_2 -------------
---------- 1-right-acc_3 -------------
---------- 1-right-acc_4 -------------
---------- 1-right-acc_5 -------------
---------- 1-right-acc_6 -------------
---------- 3-left-acc_0 -------------
---------- 3-left-acc_1 -------------
---------- 3-left-acc_2 -------------
---------- 3-left-acc_3 -------------
---------- 3-left-acc_4 -------------
---------- 3-left-acc_5 -------------
---------- 3-left-acc_6 -------------
---------- 3-right-acc_0 -------------
---------- 3-right-acc_1 -------------
---------- 3-right-acc_2 ----------

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


A total of 598 epochs were extracted and baseline corrected.
---------- 1-left-acc_0 -------------
---------- 1-left-acc_1 -------------
---------- 1-left-acc_2 -------------
---------- 1-left-acc_3 -------------
---------- 1-left-acc_4 -------------
---------- 1-left-acc_5 -------------
---------- 1-left-acc_6 -------------
---------- 1-right-acc_0 -------------
---------- 1-right-acc_1 -------------
---------- 1-right-acc_2 -------------
---------- 1-right-acc_3 -------------
---------- 1-right-acc_4 -------------
---------- 1-right-acc_5 -------------
---------- 1-right-acc_6 -------------
---------- 3-left-acc_0 -------------
---------- 3-left-acc_1 -------------
---------- 3-left-acc_2 -------------
---------- 3-left-acc_3 -------------
---------- 3-left-acc_4 -------------
---------- 3-left-acc_5 -------------
---------- 3-left-acc_6 -------------
---------- 3-right-acc_0 -------------
---------- 3-right-acc_1 -------------
---------- 3-right-acc_2 -------------
---------- 3-righ

...

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


0 bad epochs dropped
A total of 660 epochs were extracted and baseline corrected.
---------- 1-left-acc_0 -------------
---------- 1-left-acc_1 -------------
---------- 1-left-acc_2 -------------
---------- 1-left-acc_3 -------------
---------- 1-left-acc_4 -------------
---------- 1-left-acc_5 -------------
---------- 1-left-acc_6 -------------
---------- 1-right-acc_0 -------------
---------- 1-right-acc_1 -------------
---------- 1-right-acc_2 -------------
---------- 1-right-acc_3 -------------
---------- 1-right-acc_4 -------------
---------- 1-right-acc_5 -------------
---------- 1-right-acc_6 -------------
---------- 3-left-acc_0 -------------
---------- 3-left-acc_1 -------------
---------- 3-left-acc_2 -------------
---------- 3-left-acc_3 -------------
---------- 3-left-acc_4 -------------
---------- 3-left-acc_5 -------------
---------- 3-left-acc_6 -------------
---------- 3-right-acc_0 -------------
---------- 3-right-acc_1 -------------
---------- 3-right-acc_2 ----------

D:\PhD\CDA-review\studies\cda_utils.py:40: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  all_epochs = Epochs(eeg, all_events, tmin=epoch_tmin, tmax=epoch_length, baseline=baseline_corr, preload=True, event_repeated='merge')


0 bad epochs dropped
A total of 810 epochs were extracted and baseline corrected.
---------- 1-left-acc_0 -------------
---------- 1-left-acc_1 -------------
---------- 1-left-acc_2 -------------
---------- 1-left-acc_3 -------------
---------- 1-left-acc_4 -------------
---------- 1-left-acc_5 -------------
---------- 1-left-acc_6 -------------
---------- 1-right-acc_0 -------------
---------- 1-right-acc_1 -------------
---------- 1-right-acc_2 -------------
---------- 1-right-acc_3 -------------
---------- 1-right-acc_4 -------------
---------- 1-right-acc_5 -------------
---------- 1-right-acc_6 -------------
---------- 3-left-acc_0 -------------
---------- 3-left-acc_1 -------------
---------- 3-left-acc_2 -------------
---------- 3-left-acc_3 -------------
---------- 3-left-acc_4 -------------
---------- 3-left-acc_5 -------------
---------- 3-left-acc_6 -------------
---------- 3-right-acc_0 -------------
---------- 3-right-acc_1 -------------
---------- 3-right-acc_2 ----------

,1,3,6,left,right,acc_0,acc_1,acc_2,acc_3,acc_4,...,6-right-acc_4-CDA,right-acc_4-CDA,6-right-acc_5,6-right-acc_5-CDA,right-acc_5-CDA,6-right-acc_6,6-right-acc_6-CDA,right-acc_6-CDA,checksum,notes
10_DWR_Bilateral.set,230,228,230,344,344,34,269,111,233,31,...,0,0,3,0,0,0,0,0,1,
11_DWR_Bilateral.set,200,200,198,299,299,43,247,100,172,28,...,0,0,3,0,0,1,0,0,1,
12_DWR_Bilateral.set,200,201,200,301,300,24,261,152,141,20,...,0,0,2,0,0,0,0,0,1,
13_DWR_Bilateral.set,271,270,270,405,406,10,278,78,349,71,...,0,0,9,0,0,4,0,0,1,
14_DWR_Bilateral.set,196,197,196,292,297,24,258,132,162,12,...,0,0,1,0,0,0,0,0,1,
15_DWR_Bilateral.set,97,95,98,145,145,15,137,45,80,11,...,0,0,0,0,0,0,0,0,1,
16_DWR_Bilateral.set,230,230,231,345,346,29,252,114,217,56,...,0,0,12,0,0,1,0,0,1,
17_DWR_Bilateral.set,237,237,238,358,354,7,254,108,256,60,...,0,0,15,0,0,1,0,0,1,
18_DWR_Bilateral.set,200,200,200,300,300,14,250,135,175,23,...,0,0,2,0,0,0,0,0,1,
19_DWR_Bilateral.set,230,231,230,346,345,27,271,113,224,44,...,0,0,6,0,0,0,0,0,1,


In [ ]:
with open('D:\\PhD\\CDA-review\\studies\\Adam-2018-Channels.pkl', 'wb') as f:
    pickle.dump(epochs_dict_all, f)
    
del epochs_dict_all

In [4]:
with open('D:\\PhD\\CDA-review\\studies\\_channels\\Adam-2018-Channels.pkl', 'rb') as f:
    epochs_dict_all = pickle.load(f)

In [10]:
del epochs_dict_all

NameError: name 'epochs_dict_all' is not defined

In [16]:
# YR: Confirmed channels!
chan_right_all = ['PO4', 'F4', 'C4', 'P4', 'O2', 'OR', 'T4', 'T6']
chan_left_all =  ['PO3', 'F3', 'C3', 'P3', 'O1', 'OL', 'T3', 'T5']

cda_dict = dict()
cda_all_channels = dict()
for cond in conds:
    cda_all_channels[cond] = dict()
    for perf in accs:
        cda_all_channels[cond][perf] = dict()
    
for i, ch in enumerate(chan_right_all):
    for file_name in epochs_dict_all.keys():
        epochs_dict = epochs_dict_all[file_name]
        cda_dict[file_name] = dict()

        times = epochs_dict['6'][sides[0]][accs[2]].times

        for cur_cond in conds:
            cda_dict[file_name][cur_cond] = dict()

            for cur_side in sides:
                cda_dict[file_name][cur_cond][cur_side] = dict()

                for cur_perf in accs:
                    if cur_perf in epochs_dict[cur_cond][cur_side].keys():
                        if epochs_dict[cur_cond][cur_side][cur_perf] is not None:
                            epochs_right = epochs_dict[cur_cond][cur_side][cur_perf].copy().pick_channels([chan_right_all[i]])
                            epochs_left = epochs_dict[cur_cond][cur_side][cur_perf].copy().pick_channels([chan_left_all[i]])

                            evoked_right = epochs_right.average()
                            evoked_left = epochs_left.average()

                            left = evoked_left.data.mean(0)
                            right = evoked_right.data.mean(0)

                            if cur_side == 'left':
                                cda_dict[file_name][cur_cond][cur_side][cur_perf] = left - right
                            else:
                                cda_dict[file_name][cur_cond][cur_side][cur_perf] = right - left

    cda_all = dict()

    for cond in cda_dict[list(cda_dict.keys())[0]].keys():
        cda_all[cond] = dict()
        
        for perf in accs:
            cda_all[cond][perf] = dict()

            for f in cda_dict.keys():
                if (perf in cda_dict[f][cond]['left'].keys()) and (perf in cda_dict[f][cond]['right'].keys()):
                    if len(cda_dict[f][cond]['left'][perf]) > 0:
                        cda_dict_cur = (cda_dict[f][cond]['left'][perf] + cda_dict[f][cond]['right'][perf])/2
                        cda_all[cond][perf] = np.vstack([cda_all[cond][perf], cda_dict_cur]) if len(cda_all[cond][perf]) else cda_dict_cur
                    else:
                        print("==== SKIPPED: 0 points! (File: {} - Cond: {} - Perf: {})".format(str(f), str(cond), perf))
            
            if len(cda_all[cond][perf]) > 0:
                cda_all_channels[cond][perf]["{}-{}".format(chan_left_all[i], chan_right_all[i])] = cda_all[cond][perf].mean(0)

ValueError: picks (NoneNone, treated as "data_or_ica") yielded no channels, consider passing picks explicitly

In [ ]:
times = epochs_dict_all[filename][cond]['left'][perf].times

In [ ]:
cmap = matplotlib.cm.get_cmap('YlOrBr')

for perf in perfs:
    for cond in conds:
        plt.figure(figsize=(15, 12), dpi=80)
        plt.suptitle("Cond: {} [{}] - All Channel Pairs".format(cond, perf), fontsize=24)

        ch_pair_mean_amp = [cda_all_channels[cond][perf][x].mean(0) for x in cda_all_channels[cond][perf]]
        mean_list, cda_sorted = zip(*sorted(zip(ch_pair_mean_amp, cda_all_channels[cond][perf].items()), reverse=True))

        nb_pairs = len(cda_all_channels[cond][perf].keys())
        for i, ch_pair in enumerate(cda_all_channels[cond][perf].keys()):
            plt.plot((times * 1000), cda_sorted[i][1], label=cda_sorted[i][0], color=cmap(1-i/nb_pairs))

        plt.axvline(x=0, color='k', linestyle='--')
        plt.axvspan(600, 700, alpha=0.9, facecolor='lightgray')
        #plt.gca().invert_yaxis()
        plt.legend()
        plt.show()